In [1]:
pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 KB 8.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = 'ae4833097c134eda88821db7b1f6f449'
secret = 'ad313fdc0e6f4e079ad071b4adee88cf'

client_credentials_manager = SpotifyClientCredentials(client_id=cid,
                                                      client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager =
client_credentials_manager)

In [3]:
from requests import get
import pandas as pd
import re

url = 'https://www.bbc.co.uk/sounds/play/m001g0pb'
response = get(url)

tracks = []
urls = re.findall(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])', response.text)
for link in urls:
  if link[1] in ('open.spotify.com'):
    tracks.append(link[2].split('/track/')[1])

tracks = list(dict.fromkeys(tracks))

top_track_ids = []
track_heatmap_index = []
artist_name = []
song_name = []
all_track_data = []
popularity = []
track_year = []

for track in tracks:
  track_info = sp.track(track)
  track_name = track_info['name']
  artist = track_info['artists'][0]['name']
  popularity_score = track_info['popularity']
  release_year = track_info['album']['release_date']
  top_track_ids.append(track_info['id'])
  artist_name.append(artist)
  song_name.append(track_name)
  track_heatmap_index.append(f'{track_name}-{artist_name}')
  popularity.append(popularity_score)
  track_year.append(release_year)

audio_analysis = sp.audio_features(top_track_ids)
track_attributes = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

for track in audio_analysis:
  track_data = []
  for attribute in track_attributes:
    track_data.append(track[attribute])

  all_track_data.append(track_data)


df = pd.DataFrame(all_track_data,index=[song_name,artist_name],columns=track_attributes)
df.reset_index(inplace=True)
df.rename(columns={'level_0':'Track Name','level_1':'Artist'},inplace=True)
df['popularity'] = popularity
df['Release Date'] = track_year
df.sort_values(by='popularity',ascending=False,inplace=True)
print(display(df.sort_index()))

,Track Name,Artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,Release Date
0,"People, I've been sad",Christine and the Queens,0.627,0.378,-9.100,0.0382,0.2530,0.000104,0.1430,0.2210,77.982,57,2020-02-27
1,Preach,Maverick Sabre,0.348,0.127,-13.863,0.0367,0.8210,0.000000,0.1130,0.2330,136.655,34,2019-03-22
2,Feel Like Making Love,Kay Young,0.462,0.632,-9.570,0.0878,0.4030,0.000674,0.2260,0.1950,97.957,40,2022-07-08
3,In The Club (feat. Cat Burns) - Acoustic,Mahalia,0.635,0.431,-7.580,0.0525,0.8050,0.000000,0.1290,0.5270,86.933,52,2022-07-29
4,Ripples,beabadoobee,0.317,0.262,-10.469,0.0357,0.7860,0.001420,0.0995,0.3550,119.505,62,2022-07-15
5,btw,SOFY,0.408,0.251,-11.393,0.0413,0.8810,0.000000,0.6610,0.2270,168.591,39,2022-11-25
6,Stay,boerd,0.530,0.416,-9.975,0.0260,0.0524,0.012600,0.3530,0.2580,147.956,37,2022-09-30
7,ocean view,easy life,0.736,0.350,-10.424,0.2150,0.6310,0.000273,0.4100,0.3030,134.003,58,2021-05-28
8,Fairytale Lullaby,Bombay Bicycle Club,0.566,0.398,-9.778,0.0298,0.9020,0.000124,0.1170,0.5550,111.240,47,2010-01-01
9,pity season,Victoria Canal,0.612,0.301,-12.366,0.0493,0.8100,0.000008,0.1830,0.6810,134.038,33,2022-07-01


None


In [4]:
from urllib import request
from bs4 import BeautifulSoup
html = request.urlopen(url).read().decode('utf8')
soup = BeautifulSoup(html, 'html.parser')
title = soup.find('title')

oauth_scopes='user-follow-modify playlist-modify-private playlist-modify-public'


sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          client_id=cid,
          client_secret=secret,
          redirect_uri='http://localhost',
          scope=oauth_scopes, open_browser=False))

playlistresult = sp.user_playlist_create(user='mrmtoland', name=title.string)
results = sp.user_playlist_add_tracks(user='mrmtoland',playlist_id=playlistresult['id'],tracks=tracks,position=None)

Go to the following URL: https://accounts.spotify.com/authorize?client_id=ae4833097c134eda88821db7b1f6f449&response_type=code&redirect_uri=http%3A%2F%2Flocalhost&scope=user-follow-modify+playlist-modify-private+playlist-modify-public
Enter the URL you were redirected to: http://localhost/?code=AQAwpIMSMpYFNDA_87CAdvEXDsFc8u3JpJQsgGB2zk83eGIcceIJqzALI01-z1L7F6Ne0zqwF-Ghc4KefR_pMgXobg_Bk2WyGSk-anxQe46yma61QGIeY6CMt87wLdzKB9OCE0EJ5mKqvSpz6ZVXAGRyfuxNN1coBsqzdLkGveVaQ_XlvszV4rGptSliQ-QNvBuKbLv8dJUn6vweiK0KRvwqFbkF7125-QiqO1fBlswXoQJe_POLQ3U
